In [10]:
#ID:Nptpm3Kt
#FILE:rf_skf_502.csv
#SUBMITTED:~5 hours ago
#Comment 

In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/uhnigeria/Train.csv
/kaggle/input/uhnigeria/VariableDefinitions.csv
/kaggle/input/uhnigeria/SampleSubmission.csv
/kaggle/input/uhnigeria/Test.csv


# Read data 

In [12]:
# Importing the data

train_data = pd.read_csv('/kaggle/input/uhnigeria/Train.csv')
test_data = pd.read_csv('/kaggle/input/uhnigeria/Test.csv')
sample_submission = pd.read_csv('/kaggle/input/uhnigeria/SampleSubmission.csv')
variable_definition = pd.read_csv('/kaggle/input/uhnigeria/VariableDefinitions.csv')

# Simple EDA

In [13]:
# First 5 rows of the train data
train_data.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0
1,0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1
2,005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0
3,0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0
4,00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1


In [14]:
# First 5 rows of the test data
test_data.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName
0,009D84L,2010-04-24,2011-03-27,NO GENDER,25,2010-04-24,1,NaN,As Attached,Iveco,Victoria Island,Lagos,CVTP
1,01DO2EQ,2010-01-01,2010-12-31,NaN,120,2010-01-01,4,NaN,As Attached,TOYOTA,Victoria Island,Lagos,Muuve
2,01QM0NU,2010-10-23,2011-10-22,Female,46,2010-10-23,1,NaN,NaN,Ford,Abuja Municipal,Abuja-Municipal,Car Classic
3,024NJLZ,2010-10-14,2011-10-13,Male,32,2010-10-14,1,NaN,NaN,NaN,Kosofe,Benue,Car Classic
4,02BYET3,2010-09-16,2010-12-31,NaN,120,2010-09-16,4,NaN,NaN,TOYOTA,Victoria Island,Lagos,Muuve


In [15]:
# Last 5 rows of the test data
test_data.tail()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName
5172,ZYXX5AF,2010-07-18,2011-07-17,Male,48,2010-07-18,1,Saloon,NaN,Honda,NaN,NaN,CarSafe
5173,ZYYOZ5L,2010-12-04,2011-12-03,NaN,50,2010-12-04,1,NaN,NaN,NaN,Victoria Island,Lagos,Car Classic
5174,ZZ1GTKD,2010-09-24,2011-09-23,Male,1,2010-09-24,1,NaN,NaN,NaN,NaN,NaN,CarSafe
5175,ZZDXQSI,2010-07-17,2011-07-16,Female,52,2010-07-17,1,Saloon,Grey,TOYOTA,NaN,NaN,Car Classic
5176,ZZYTLV1,2010-07-17,2011-07-16,Male,68,2010-07-17,1,Saloon,NaN,Peugeot,NaN,NaN,CarSafe


In [16]:
# First 5 rows of the Sample SUbmission file
sample_submission.head()

,ID,target
0,009D84L,0
1,01DO2EQ,0
2,01QM0NU,0
3,024NJLZ,0
4,02BYET3,0


In [17]:
# First 5 rows of the Variable definition file
variable_definition.head()

,ID,Unique ID for the customer
0,Policy Start Date,Date policy started
1,Policy End Date,Date policy ended
2,Gender,Gender of the customer
3,Age,Age of the customer
4,ProductName,Name of Insurance policy


In [18]:
# Shapes of the train_data, test_data, sample_submission file 
print('Train shape:',train_data.shape,'\nTest shape:', test_data.shape, '\nsample_submission shape:',sample_submission.shape)

Train shape: (12079, 14) 
Test shape: (5177, 13) 
sample_submission shape: (5177, 2)


In [19]:
# Train_data info
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12079 entries, 0 to 12078
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      12079 non-null  object
 1   Policy Start Date       12079 non-null  object
 2   Policy End Date         12079 non-null  object
 3   Gender                  11720 non-null  object
 4   Age                     12079 non-null  int64 
 5   First Transaction Date  12079 non-null  object
 6   No_Pol                  12079 non-null  int64 
 7   Car_Category            8341 non-null   object
 8   Subject_Car_Colour      5117 non-null   object
 9   Subject_Car_Make        9603 non-null   object
 10  LGA_Name                5603 non-null   object
 11  State                   5591 non-null   object
 12  ProductName             12079 non-null  object
 13  target                  12079 non-null  int64 
dtypes: int64(3), object(11)
memory usage: 1.3+ MB


In [20]:
# test_data info
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5177 entries, 0 to 5176
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ID                      5177 non-null   object
 1   Policy Start Date       5177 non-null   object
 2   Policy End Date         5177 non-null   object
 3   Gender                  5021 non-null   object
 4   Age                     5177 non-null   int64 
 5   First Transaction Date  5177 non-null   object
 6   No_Pol                  5177 non-null   int64 
 7   Car_Category            3539 non-null   object
 8   Subject_Car_Colour      2172 non-null   object
 9   Subject_Car_Make        4116 non-null   object
 10  LGA_Name                2395 non-null   object
 11  State                   2389 non-null   object
 12  ProductName             5177 non-null   object
dtypes: int64(2), object(11)
memory usage: 525.9+ KB


# Concatenation of the test and train dataset

In [21]:
# join train and test together for easy preprocessing
num_train = train_data.shape[0]
num_test = test_data.shape[0]

data = pd.concat((train_data, test_data)).reset_index(drop=True)
print("data size is : {}".format(data.shape))

data size is : (17256, 14)


In [22]:
# Unique values for number of policies
data.No_Pol.unique()

array([ 1,  3,  2,  4,  5,  7, 10,  6])

In [23]:
# Unique values for ProductNames
data.ProductName.unique()

array(['Car Classic', 'CarSafe', 'Muuve', 'CVTP', 'Car Plus',
       'Motor Cycle', 'Customized Motor', 'CarFlex', 'Car Vintage'],
      dtype=object)

In [24]:
# Datetime columns
date_columns = ['Policy Start Date','Policy End Date','First Transaction Date']
# Numeric columns
num_columns = ['Age']
# Categorical columns
categorical_col = [col for col in test_data.columns if col not in date_columns+num_columns]
categorical_col

['ID',
 'Gender',
 'No_Pol',
 'Car_Category',
 'Subject_Car_Colour',
 'Subject_Car_Make',
 'LGA_Name',
 'State',
 'ProductName']

In [25]:
# Removing ID column from the categorical column list
categorical_col.remove('ID')

In [26]:
# Statistical Description of the train data
train_data.describe()

,Age,No_Pol,target
count,12079.000000,12079.000000,12079.000000
mean,42.234539,1.307227,0.120457
std,97.492565,0.733085,0.325509
min,-6099.000000,1.000000,0.000000
25%,35.000000,1.000000,0.000000
50%,41.000000,1.000000,0.000000
75%,50.000000,1.000000,0.000000
max,320.000000,10.000000,1.000000


In [27]:
# Statistical Description of the test data
test_data.describe()

,Age,No_Pol
count,5177.000000,5177.000000
mean,43.710257,1.308287
std,22.439018,0.712004
min,-76.000000,1.000000
25%,34.000000,1.000000
50%,40.000000,1.000000
75%,50.000000,1.000000
max,320.000000,10.000000


In [28]:
# Percentage of null values in the train data
print((train_data.isnull().sum()/train_data.shape[0])*100)

ID                         0.000000
Policy Start Date          0.000000
Policy End Date            0.000000
Gender                     2.972100
Age                        0.000000
First Transaction Date     0.000000
No_Pol                     0.000000
Car_Category              30.946270
Subject_Car_Colour        57.637222
Subject_Car_Make          20.498386
LGA_Name                  53.613710
State                     53.713056
ProductName                0.000000
target                     0.000000
dtype: float64


In [29]:
# Percentage of null values in the test data
print((test_data.isnull().sum()/test_data.shape[0])*100)

ID                         0.000000
Policy Start Date          0.000000
Policy End Date            0.000000
Gender                     3.013328
Age                        0.000000
First Transaction Date     0.000000
No_Pol                     0.000000
Car_Category              31.639946
Subject_Car_Colour        58.045200
Subject_Car_Make          20.494495
LGA_Name                  53.737686
State                     53.853583
ProductName                0.000000
dtype: float64


In [30]:
# First 5 rows of all the data
data.head()

,ID,Policy Start Date,Policy End Date,Gender,Age,First Transaction Date,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,target
0,0040R73,2010-05-14,2011-05-13,Male,30,2010-05-14,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,0.0
1,0046BNK,2010-11-29,2011-11-28,Female,79,2010-11-29,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,1.0
2,005QMC3,2010-03-21,2011-03-20,Male,43,2010-03-21,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,0.0
3,0079OHW,2010-08-21,2011-08-20,Male,2,2010-08-21,1,NaN,NaN,NaN,NaN,NaN,CarSafe,0.0
4,00BRP63,2010-08-29,2010-12-31,Entity,20,2010-08-29,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,1.0


In [31]:
# Unique values of the Gender columns
data.Gender.unique()#checking for unique sex

array(['Male', 'Female', 'Entity', 'Joint Gender', nan, 'NO GENDER',
       'NOT STATED', 'SEX'], dtype=object)

# Basic Data preprocessing 


# filling in mising value

In [32]:
# Percentage of null values in the entire data
print((data.isnull().sum()/data.shape[0])*100)

ID                         0.000000
Policy Start Date          0.000000
Policy End Date            0.000000
Gender                     2.984469
Age                        0.000000
First Transaction Date     0.000000
No_Pol                     0.000000
Car_Category              31.154381
Subject_Car_Colour        57.759620
Subject_Car_Make          20.497218
LGA_Name                  53.650904
State                     53.755216
ProductName                0.000000
target                    30.001159
dtype: float64


###  Converting date features to datetime

In [33]:
date_columns

['Policy Start Date', 'Policy End Date', 'First Transaction Date']

In [34]:
# Converting dates to datetime colunms
for date in date_columns:
    data[date] = pd.to_datetime(data[date])

In [35]:
# Checking info on the entire data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17256 entries, 0 to 17255
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID                      17256 non-null  object        
 1   Policy Start Date       17256 non-null  datetime64[ns]
 2   Policy End Date         17256 non-null  datetime64[ns]
 3   Gender                  16741 non-null  object        
 4   Age                     17256 non-null  int64         
 5   First Transaction Date  17256 non-null  datetime64[ns]
 6   No_Pol                  17256 non-null  int64         
 7   Car_Category            11880 non-null  object        
 8   Subject_Car_Colour      7289 non-null   object        
 9   Subject_Car_Make        13719 non-null  object        
 10  LGA_Name                7998 non-null   object        
 11  State                   7980 non-null   object        
 12  ProductName             17256 non-null  object

In [36]:
# Function to extract date information
def extract_date_info(df,cols,):
    for feat in cols:
        df[feat +'_year'] = df[feat].dt.quarter
        df[feat +'_day'] = df[feat].dt.day
        df[feat +'_month'] = df[feat].dt.month
        df[feat +'_quarter'] = df[feat].dt.quarter
    df.drop(columns=date_columns,axis=1,inplace=True)

In [37]:
# Extracting the date information in the entire data
extract_date_info(data,date_columns)

In [38]:
data.head()

,ID,Gender,Age,No_Pol,Car_Category,Subject_Car_Colour,Subject_Car_Make,LGA_Name,State,ProductName,...,Policy Start Date_month,Policy Start Date_quarter,Policy End Date_year,Policy End Date_day,Policy End Date_month,Policy End Date_quarter,First Transaction Date_year,First Transaction Date_day,First Transaction Date_month,First Transaction Date_quarter
0,0040R73,Male,30,1,Saloon,Black,TOYOTA,NaN,NaN,Car Classic,...,5,2,2,13,5,2,2,14,5,2
1,0046BNK,Female,79,1,JEEP,Grey,TOYOTA,NaN,NaN,Car Classic,...,11,4,4,28,11,4,4,29,11,4
2,005QMC3,Male,43,1,Saloon,Red,TOYOTA,NaN,NaN,Car Classic,...,3,1,1,20,3,1,1,21,3,1
3,0079OHW,Male,2,1,NaN,NaN,NaN,NaN,NaN,CarSafe,...,8,3,3,20,8,3,3,21,8,3
4,00BRP63,Entity,20,3,NaN,NaN,NaN,Lagos,Lagos,Muuve,...,8,3,4,31,12,4,3,29,8,3


In [39]:
# Unique gender values
data.Gender.unique()

array(['Male', 'Female', 'Entity', 'Joint Gender', nan, 'NO GENDER',
       'NOT STATED', 'SEX'], dtype=object)

In [40]:
# A dictionary to map gender values
gender_map = {"Male":"M","Female":'F','Entity':'O','Joint Gender':'O',9999:'O','NO GENDER':'O','NOT STATED':'O','SEX':'O' }

In [41]:
# Mapping gender values in the data
data.Gender = data.Gender.map(gender_map)

In [42]:
data.No_Pol

0        1
1        1
2        1
3        1
4        3
        ..
17251    1
17252    1
17253    1
17254    1
17255    1
Name: No_Pol, Length: 17256, dtype: int64

#  Filling missing values in the dataset

In [43]:
data['Gender'] = data.Gender.fillna('9999')
data['No_Pol'] = data.No_Pol.fillna('9999')
data['Car_Category'] = data.Car_Category.fillna('9999')
data['Subject_Car_Colour'] = data.Subject_Car_Colour.fillna('9999')
data['Subject_Car_Make'] = data.Subject_Car_Make.fillna('9999')
data['LGA_Name'] = data.LGA_Name.fillna('9999')
data['State'] = data.State.fillna('9999')
data['ProductName'] = data.ProductName.fillna('9999')

# Checking for unique value in categorical columns

In [44]:
data.State.unique()

array(['9999', 'Lagos', 'Benue', 'Eti-Osa', 'Delta', 'Ogun',
       'Abuja-Municipal', 'Oshodi-Isolo', 'Ibeju-Lekki', 'Ibadan-West',
       'Obia-Akpor', 'Ibadan-East', 'Kaduna-South', 'Amuwo-Odofin',
       'Anambra', 'Rivers', 'Niger-State', 'Edo', 'Akwa-Ibom',
       'ENUGU-EAST', 'Abuja', 'Kaduna', 'Esan-West', 'Obafemi-Owode',
       'Orile-Iganmu', 'Ifako-Ijaye', 'Ibadan-North', 'Imo', 'Jos-North',
       'Ondo-West', 'Kaduna-North', 'Aniocha-South', 'Awka-South',
       'Ile-Oluji', 'Ijebu-Ode', 'Port-Harcourt', 'Nnewi-North',
       'Aboh-Mbaise', 'Oyo', 'Akoko-West', 'Warri-Central', 'Cross-River',
       'Jos-South', 'Onitsha-North', 'Osun', 'Ajeromi-Ifelodun',
       'Kano-Municipal', 'Ife-Central', 'Ilorin-West', 'Lagelu-North',
       'Owerri-Municipal', 'Nsit-Ubium', 'Udi-Agwu', 'Essien-Udim',
       'Owerri-West', 'Ondo', 'Ogbmosho-South', 'Umuahia-South',
       'ABULE-EGBA', 'Enugu-North', 'Ovia-SouthWest', 'Kebbi',
       'Ajegunle-State', 'Ekiti-West', 'Ogun-Watersid

In [45]:
data.LGA_Name.unique()

array(['9999', 'Lagos', 'Ikeja', 'Badagry', 'Eti-Osa', 'Victoria Island',
       'Ikoyi', 'Lagos Mainland', 'EFFURUN', 'Abeokuta',
       'Abuja Municipal', 'Yaba', 'Aguda', 'Surulere', 'Oshodi-Isolo',
       'Alimosho', 'Okpe, Delta State', 'Asaba', 'IbadanCentral', 'Lekki',
       'Ibeju/Lekki', 'OBANIKORO', 'Agege', 'ISHERI', 'Kosofe', 'OGUN',
       'Apapa', 'Ibadan South West', 'Obia/Akpor', 'Ibadan South East',
       'Sapele', 'Kaduna South', 'Ikorodu', 'Amuwo-Odofin', 'Awka',
       'GBAGADA', 'Port Harcourt', 'Minna', 'Shomolu', 'Benin',
       'Lagos Island', 'Central', 'Ebute-Metta', 'Ketu', 'Uyo', 'Mushin',
       'Katagum', 'ENUGU EAST', 'Oredo', 'Abuja', 'Festac', 'Ipaja',
       'Zaria ', 'Esan West', 'Oluyole', 'Bwari', 'Obafemi-Owode', 'LGA',
       'Orile-Iganmu', 'Egbeda', 'Ido', 'Ifako-Ijaye ', 'Ibadan North',
       'Oguta', 'Jos North', 'Ajah', 'Ondo West', 'Kaduna North', 'Warri',
       'Aniocha South', 'Owerri', 'Awka South', 'Asokoro District',
       'Ile-Olu

In [46]:
data.Subject_Car_Make.unique()

array(['TOYOTA', '9999', 'REXTON', 'Lexus', 'Hyundai', 'Iveco', 'DAF',
       'Honda', 'Mercedes', 'Jincheng', 'ACURA', 'Ford', 'Volkswagen',
       'Nissan', '.', 'Pontiac', 'Range Rover', 'Kia', 'Mitsubishi',
       'Scania', 'BMW', 'Infiniti', 'Renault', 'Volvo', 'Hummer', 'Mack',
       'Grand Cherokee', 'Porsche', 'Peugeot', 'Land Rover', 'Chevrolet',
       'Mazda', 'Man', 'Jeep', 'Audi', 'Suzuki', 'MINI COOPER', 'KA',
       'As Attached', 'Innson', 'Isuzu', 'Skoda', 'Jaguar', 'Chrysler',
       'Dodge', 'GMC', 'Land Rover.', 'Subaru', 'GAC', 'Fiat', 'Opel',
       'Astra', 'Motorcycle', 'ZOYTE', 'Seat', 'FOTON', 'Howo', 'Lincoln',
       'Rols Royce', 'CHANGAN', 'LIBERTY', 'ABG', 'Yamaha', 'Black',
       'Tata', 'Ashok Leyland', 'Geely', 'Caddillac', 'Wrangler Jeep',
       'Raston', 'MG', 'BRILLIANCE', 'Buik', 'COMMANDER', 'Bajaj',
       'Datsun'], dtype=object)

In [47]:
data.ProductName.unique()

array(['Car Classic', 'CarSafe', 'Muuve', 'CVTP', 'Car Plus',
       'Motor Cycle', 'Customized Motor', 'CarFlex', 'Car Vintage'],
      dtype=object)

In [48]:
data.Subject_Car_Make.unique()

array(['TOYOTA', '9999', 'REXTON', 'Lexus', 'Hyundai', 'Iveco', 'DAF',
       'Honda', 'Mercedes', 'Jincheng', 'ACURA', 'Ford', 'Volkswagen',
       'Nissan', '.', 'Pontiac', 'Range Rover', 'Kia', 'Mitsubishi',
       'Scania', 'BMW', 'Infiniti', 'Renault', 'Volvo', 'Hummer', 'Mack',
       'Grand Cherokee', 'Porsche', 'Peugeot', 'Land Rover', 'Chevrolet',
       'Mazda', 'Man', 'Jeep', 'Audi', 'Suzuki', 'MINI COOPER', 'KA',
       'As Attached', 'Innson', 'Isuzu', 'Skoda', 'Jaguar', 'Chrysler',
       'Dodge', 'GMC', 'Land Rover.', 'Subaru', 'GAC', 'Fiat', 'Opel',
       'Astra', 'Motorcycle', 'ZOYTE', 'Seat', 'FOTON', 'Howo', 'Lincoln',
       'Rols Royce', 'CHANGAN', 'LIBERTY', 'ABG', 'Yamaha', 'Black',
       'Tata', 'Ashok Leyland', 'Geely', 'Caddillac', 'Wrangler Jeep',
       'Raston', 'MG', 'BRILLIANCE', 'Buik', 'COMMANDER', 'Bajaj',
       'Datsun'], dtype=object)

In [49]:
data.ProductName.unique()

array(['Car Classic', 'CarSafe', 'Muuve', 'CVTP', 'Car Plus',
       'Motor Cycle', 'Customized Motor', 'CarFlex', 'Car Vintage'],
      dtype=object)

In [50]:
data['Subject_Car_Make'].unique()

array(['TOYOTA', '9999', 'REXTON', 'Lexus', 'Hyundai', 'Iveco', 'DAF',
       'Honda', 'Mercedes', 'Jincheng', 'ACURA', 'Ford', 'Volkswagen',
       'Nissan', '.', 'Pontiac', 'Range Rover', 'Kia', 'Mitsubishi',
       'Scania', 'BMW', 'Infiniti', 'Renault', 'Volvo', 'Hummer', 'Mack',
       'Grand Cherokee', 'Porsche', 'Peugeot', 'Land Rover', 'Chevrolet',
       'Mazda', 'Man', 'Jeep', 'Audi', 'Suzuki', 'MINI COOPER', 'KA',
       'As Attached', 'Innson', 'Isuzu', 'Skoda', 'Jaguar', 'Chrysler',
       'Dodge', 'GMC', 'Land Rover.', 'Subaru', 'GAC', 'Fiat', 'Opel',
       'Astra', 'Motorcycle', 'ZOYTE', 'Seat', 'FOTON', 'Howo', 'Lincoln',
       'Rols Royce', 'CHANGAN', 'LIBERTY', 'ABG', 'Yamaha', 'Black',
       'Tata', 'Ashok Leyland', 'Geely', 'Caddillac', 'Wrangler Jeep',
       'Raston', 'MG', 'BRILLIANCE', 'Buik', 'COMMANDER', 'Bajaj',
       'Datsun'], dtype=object)

# Using LabelEncoder to encode the categorical variables

In [51]:
from sklearn.preprocessing import LabelEncoder
le =LabelEncoder()


In [52]:
data['Subject_Car_Make']=le.fit_transform(data['Subject_Car_Make'])
data['LGA_Name']=le.fit_transform(data['LGA_Name'])
data['State']=le.fit_transform(data['State'])
data['ProductName']=le.fit_transform(data['ProductName'])


In [53]:
data['Gender']=le.fit_transform(data['Gender'])
data['No_Pol']=le.fit_transform(data['No_Pol'])
data['Subject_Car_Colour']=le.fit_transform(data['Subject_Car_Colour'])
data['Car_Category']=le.fit_transform(data['Car_Category'])

In [54]:
print((data.isnull().sum()/data.shape[0])*100)

ID                                 0.000000
Gender                             0.000000
Age                                0.000000
No_Pol                             0.000000
Car_Category                       0.000000
Subject_Car_Colour                 0.000000
Subject_Car_Make                   0.000000
LGA_Name                           0.000000
State                              0.000000
ProductName                        0.000000
target                            30.001159
Policy Start Date_year             0.000000
Policy Start Date_day              0.000000
Policy Start Date_month            0.000000
Policy Start Date_quarter          0.000000
Policy End Date_year               0.000000
Policy End Date_day                0.000000
Policy End Date_month              0.000000
Policy End Date_quarter            0.000000
First Transaction Date_year        0.000000
First Transaction Date_day         0.000000
First Transaction Date_month       0.000000
First Transaction Date_quarter  

In [55]:
# filling missing values in the target column due to the concatetion of both the training and test dataset
data.target = data.target.fillna(9999)

In [56]:
#Changing all the column.dtypes to integer
data.target = data.target.astype(int)

In [57]:
#Dropping unnecessary columns
data.drop(columns=['ID'],inplace=True)

In [58]:
#Get the new dataset
train_new = data[:num_train]
test_new = data[num_train:]

In [59]:
test_new.drop("target",axis = 1,inplace = True)#dropping the target column in the test dataset

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [60]:
X= train_new.drop(columns=['target']) #features to train on
y= train_new.target #Target variable

# Building a model

In [61]:
# Importing our model
from sklearn.ensemble import RandomForestClassifier

In [62]:
# F1_score for Checking the accuracy score of the model
from sklearn.metrics import f1_score

In [63]:
# for cross validation on the dataset
from sklearn.model_selection import StratifiedKFold

In [64]:
seed = 102 #Seed value of 102

skf = StratifiedKFold(n_splits=25, shuffle=True, random_state=seed)  # Stratified kfold with 25 splits

scores = [] # a list to store our validation scores

preds = []  # a list to store our prediction per splits

rforest = RandomForestClassifier(n_jobs=-1,random_state =102, n_estimators=500) # Our Model, RandomForestClassifier with seed value 102, and n_estimator value of 500

# A For loop to train and make predictions on every split generated by the StratifiedKFold

i = 1
for train, test in skf.split(X, y):
    x_train, x_test, y_train, y_test = X.iloc[train], X.iloc[test], y.iloc[train], y.iloc[test]
    rforest.fit(x_train, y_train)
    print('Number of splits trained {} '.format(i))
    score = f1_score(y_test, rforest.predict(x_test))
    pred = rforest.predict_proba(test_new)[:, 1]
    scores.append(score)
    preds.append(pred)
    i += 1

np.mean(scores) # The mean of our validation scores

Number of splits trained 1 
Number of splits trained 2 
Number of splits trained 3 
Number of splits trained 4 
Number of splits trained 5 
Number of splits trained 6 
Number of splits trained 7 
Number of splits trained 8 
Number of splits trained 9 
Number of splits trained 10 
Number of splits trained 11 
Number of splits trained 12 
Number of splits trained 13 
Number of splits trained 14 
Number of splits trained 15 
Number of splits trained 16 
Number of splits trained 17 
Number of splits trained 18 
Number of splits trained 19 
Number of splits trained 20 
Number of splits trained 21 
Number of splits trained 22 
Number of splits trained 23 
Number of splits trained 24 
Number of splits trained 25 


0.177990425850567

# Making a submission file

In [66]:
preds_mean = np.mean(preds, axis=0) # The average of our predictions per row

final = [] # A list to store our final predicitions

""" We trained our model in a StratifiedKFold and made probability predicitions per split, to get our final prediction, we go ahead to set a threshold of 0.4, 
where if our probability value of a claim being made is greater than 0.4, our final prediction is set to 1, else it is set to 0"""

for x in preds_mean:
    if x > 0.4:
        final.append(1)
    else:
        final.append(0)

# Creating our submission file
submit = sample_submission.copy()
submit.target = final
submit.to_csv('rf_skf_502.csv',index=False)

**Notebook and Submission file made by team: Cruise_fc. 
Placed 6th position on the leaderboard
Winning 5th place for Federal University of Agriculture Abeokuta.**